In [127]:
import torch
from stable_baselines3 import PPO
import gymnasium as gym
import numpy as np
import imageio
import subprocess
import shutil
import time
import os

In [128]:
class CustomRewardWrapper(gym.Wrapper):
    def __init__(self, env):
        super(CustomRewardWrapper, self).__init__(env)

    def step(self, action):
        obs, reward, done, truncated, info = self.env.step(action)
        # Access relevant state and action variables
        
        #state
        z = obs[0]                  # Height of the robot
        a = obs[1]                  # Angle of the robot
        a_hip = obs[2]              # Angle of the hip
        a_knee = obs[3]             # Angle of the knee
        a_ankle = obs[4]            # Angle of the ankle
        v_x = obs[5]                # Velocity in x direction
        v_z = obs[6]                # Velocity in z direction
        a_d = obs[7]                # Angular velocity 
        a_hip_d = obs[8]            # Angular velocity of the hip
        a_knee_d = obs[9]           # Angular velocity of the knee
        a_ankle_d = obs[10]         # Angular velocity of the ankle
        
        #action
        torque_hip = action[0]      # Torque applied to the hip
        torque_knee = action[1]     # Torque applied to the knee
        torque_ankle = action[2]    # Torque applied to the ankle

        #different criteria for reward
        energy_used = np.sum(np.square(action))  # Simplistic energy calculation

        # Custom reward logic
        custom_reward = z * 0.5                                     # Reward for height
        custom_reward += np.tanh(a) * 0.5                           # Reward for positive angles
        custom_reward += 5*np.exp(-(a-(2*np.pi))**2/(2*np.pi))      # Reward for 2pi angle
        custom_reward -= energy_used * 0.01                         # Penalize energy consumption

        # used by openai
        # backroll = -obs[7]
        # height = obs[0]
        # vel_act = action[0] * obs[8] + action[1] * obs[9] + a[2] * obs[10]
        # backslide = -obs[5]
        # reward = backroll * (1.0 + .3 * height + .1 * vel_act + .05 * backslide)

        if done:
            custom_reward -= 8  # Heavy penalty for falling

        return obs, custom_reward, done, truncated, info

In [129]:
healthy_reward = 0.8
healthy_z_range = (0.2, float("inf"))
healthy_angle_range = (-np.pi/4, 2*np.pi+np.pi/4)
reset_noise_scale = 5e-3
exclude_current_positions_from_observation = True

env = gym.make('Hopper-v4', render_mode='rgb_array', healthy_reward=healthy_reward, healthy_z_range=healthy_z_range, healthy_angle_range=healthy_angle_range, reset_noise_scale=reset_noise_scale, exclude_current_positions_from_observation=exclude_current_positions_from_observation)
env = CustomRewardWrapper(env)

model = PPO("MlpPolicy", env, verbose=1)

# Train the model
n_learning_steps = 30000
model.learn(total_timesteps=n_learning_steps)

# Save the model
model.save("hopper")

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 45.1     |
|    ep_rew_mean     | 12.5     |
| time/              |          |
|    fps             | 959      |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 49.7        |
|    ep_rew_mean          | 15          |
| time/                   |             |
|    fps                  | 522         |
|    iterations           | 2           |
|    time_elapsed         | 7           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.011493967 |
|    clip_fraction        | 0.132       |
|    clip_range           | 0.2         |
|    entropy_loss   

In [130]:
vec_env = model.get_env()
obs = vec_env.reset()

writer = imageio.get_writer('hopper-flip.mp4', fps=100)

N_step = 100000
for i in range(N_step):
    action, _state = model.predict(obs, deterministic=True)
    obs, reward, done, info = vec_env.step(action)
    print(obs[0][1])
    writer.append_data(vec_env.render("rgb_array"))
    #VecEnv resets automatically
    if done:
      print("Episode finished after {} timesteps".format(i+1))
      break
      obs = vec_env.reset()

writer.close()

# d = time.strftime("%Y-%m-%d-%H-%M-%S")
# current_py_file = os.getcwd() + '/hopper.ipynb'
# new_py_file = os.getcwd() + '/Run/hopper%s.ipynb'%(d)

# shutil.copyfile(current_py_file, new_py_file)
# shutil.copyfile('hopper-flip.mp4', os.getcwd()+'/Render/hopper-flip%s.mp4'%(d))

0.004203217408811462
0.005647509451345368
0.006314052537679126
0.0066377236869957215
0.006849729718792066
0.007066359671181791
0.007340097298979606
0.0076887242706275925
0.008182330530233067
0.009799980680844167
0.012406924064512583
0.01559505752496649
0.019095783543318017
0.022720045581553522
0.026328720252710054
0.02981758284044927
0.03310944422579978
0.03615029733571118
0.038907688412827164
0.041378273742380865
0.04359207784554041
0.045581018938305455
0.047372585621378534
0.04898525351611534
0.05043101407509326
0.051723032094286865
0.05245039606398572
0.05250884942100792
0.05224676461160573
0.05191024222177236
0.05166780635195523
0.051627218312042505
0.05184752304386134
0.052339014219668564
0.05302914589620967
0.0539047200769457
0.05531061924900315
0.057198402637849546
0.059474608308042645
0.06205888490306764
0.06487751977244623
0.06786117979020931
0.07094461776733993
0.07406719411528902
0.07717365717928447
0.08021495745173597
0.08314906596352747
0.08594186552708927
0.08856816459844

In [131]:
np.pi/2

1.5707963267948966